In [1]:
import os
import pandas as pd
from pathlib import Path
from moviepy.editor import VideoFileClip

In [2]:
def extract_segments(csv_path, video_path, output_dir):
    df = pd.read_csv(csv_path)

    people_counts = df['People Count'].tolist()
    times = df['Time (s)'].tolist()

    segments = []
    start_idx = None
    people_streak = 0
    zero_streak = 0

    for i in range(len(people_counts)):
        if people_counts[i] > 0:
            people_streak += 1
            zero_streak = 0
        else:
            zero_streak += 1
            if zero_streak > 5:
                if start_idx is not None:
                    segments.append((times[start_idx], times[i]))
                    start_idx = None
                zero_streak = 0
                people_streak = 0

        if people_streak > 5 and start_idx is None:
            start_idx = i - 5

    if start_idx is not None:
        segments.append((times[start_idx], times[-1]))

    if not segments:
        return

    os.makedirs(output_dir, exist_ok=True)
    base_name = Path(video_path).stem

    with VideoFileClip(video_path) as video:
        for idx, (start, end) in enumerate(segments, 1):
            output_file = os.path.join(output_dir, f"{base_name}_{idx}.webm")

            clip = video.subclip(start, end)
            clip.write_videofile(output_file, codec='libvpx', audio_codec='libvorbis', verbose=False, logger=None)
            print(f"Saved segment: {output_file}")

In [ ]:
def main():
    recordings_dir = "recordings"
    timestamps_dir = "timestamps"
    cropped_dir = "cropped"

    for cam in range(1, 6):
        cam_folder = f"camera{cam}"
        cam_path = os.path.join(recordings_dir, cam_folder)
        out_path = os.path.join(cropped_dir, cam_folder)
        timestamp_path = os.path.join(timestamps_dir, cam_folder)

        if not os.path.exists(cam_path) or not os.path.exists(timestamp_path):
            continue

        for file in os.listdir(cam_path):
            if file.endswith(".mp4"):
                video_path = os.path.join(cam_path, file)
                csv_path = os.path.join(timestamp_path, Path(file).stem + ".csv")

                if os.path.exists(csv_path):
                    extract_segments(csv_path, video_path, out_path)


if __name__ == "__main__":
    main()

Saved segment: cropped/camera1/camera1_20250317_043858_1.webm
Saved segment: cropped/camera1/camera1_20250317_094552_1.webm
Saved segment: cropped/camera1/camera1_20250317_094552_2.webm
Saved segment: cropped/camera1/camera1_20250317_094552_3.webm
Saved segment: cropped/camera1/camera1_20250317_094552_4.webm
Saved segment: cropped/camera1/camera1_20250317_094552_5.webm
Saved segment: cropped/camera1/camera1_20250317_094552_6.webm
Saved segment: cropped/camera1/camera1_20250317_094552_7.webm
Saved segment: cropped/camera1/camera1_20250317_094552_8.webm
